# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Import Clean City Data
weather = os.path.join('..', 'WheaterPy','Output_Data','Clean_City_Data.csv')
weather_df = pd.read_csv(weather)
weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date
0,Rikitea,-23.1203,-134.9692,22.71,70,95,7.24,PF,1622095617
1,Saldanha,-33.0117,17.9442,14.21,84,98,1.83,ZA,1622095746
2,Chui,-33.6971,-53.4616,11.64,87,31,5.02,UY,1622095747
3,Hobart,-42.8794,147.3294,6.71,75,75,0.89,AU,1622095600
4,Arraial do Cabo,-22.9661,-42.0278,21.58,89,100,3.43,BR,1622095748
...,...,...,...,...,...,...,...,...,...
556,Hueyapan,18.8667,-98.6667,15.85,77,95,1.40,MX,1622095934
557,Tirano,46.2154,10.1679,14.36,56,35,1.30,IT,1622095934
558,Camacha,33.0833,-16.3333,16.67,68,75,4.63,PT,1622095935
559,Chake Chake,-5.2459,39.7666,26.49,75,20,6.13,TZ,1622095935


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Store latitude and longitude in locations
world_location = weather_df[['Lat', 'Lng']]

# Store humidity
humidity = weather_df['Humidity (%)']

In [11]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(world_location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
fitting_weather = weather_df.loc[(weather_df['Max Temp (C)']>=18) & (weather_df['Max Temp (C)']<=33) & (weather_df['Wind Speed (kph)']<=18) & (weather_df['Cloudiness (%)'] == 0),:]
fitting_weather.reset_index(drop=True)

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date
0,Miandrivazo,-19.5167,45.4667,23.45,57,0,0.90,MG,1622095634
1,Sawtell,-30.3833,153.1000,19.01,55,0,4.12,AU,1622095642
2,Miepherkeim,38.1419,41.0081,22.16,23,0,1.20,TR,1622095642
3,Mossamedes,-15.1961,12.1522,23.25,58,0,0.96,AO,1622095665
4,San Juan,-31.5375,-68.5364,21.52,36,0,0.76,AR,1622095419
5,São Filipe,14.8961,-24.4956,22.93,77,0,4.67,CV,1622095680
6,Gachsaran,30.3586,50.7981,31.38,13,0,1.75,IR,1622095795
7,Sakakah,29.9697,40.2064,32.25,11,0,3.12,SA,1622095808
8,Saint-Pierre,-21.3393,55.4781,25.90,68,0,1.54,RE,1622095820
9,Oruzgan,32.8333,66.0000,29.41,9,0,1.08,AF,1622095829


In [27]:
# fitting_weather_df = fitting_weather.drop(fitting_weather.index[24])
# fitting_weather_df.reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# hotel_df = []
# # target_search = 'hotel'
# target_radius = 5000
# target_type = "lodging"

# for city in range(len(fitting_weather_df['City'])):
#     lat = fitting_weather_df.iloc[city]['Lat']
#     lng = fitting_weather_df.iloc[city]['Lng']
#     coords = f'{lat},{lng}'
#     params = {
#     "location": coords,
# #     "keyword": target_search,
#     "radius": target_radius,
#     "type": target_type,
#     "key": g_key}
#     # base url
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     # run a request using our params dictionary
#     response = requests.get(base_url, params=params).json()
#     try:
#         hotel_df.append(response["results"][0]["name"])    
#     except:
#         hotel_df.append(''No lodging nearby'')

In [28]:
# Store into variable named hotel_df
hotel_df = fitting_weather
hotel_df['Hotel Name'] = ''
hotel_df

<ipython-input-28-592a00adefcc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date,Hotel Name
47,Miandrivazo,-19.5167,45.4667,23.45,57,0,0.90,MG,1622095634,
67,Sawtell,-30.3833,153.1000,19.01,55,0,4.12,AU,1622095642,
68,Miepherkeim,38.1419,41.0081,22.16,23,0,1.20,TR,1622095642,
128,Mossamedes,-15.1961,12.1522,23.25,58,0,0.96,AO,1622095665,
134,San Juan,-31.5375,-68.5364,21.52,36,0,0.76,AR,1622095419,
163,São Filipe,14.8961,-24.4956,22.93,77,0,4.67,CV,1622095680,
221,Gachsaran,30.3586,50.7981,31.38,13,0,1.75,IR,1622095795,
242,Sakakah,29.9697,40.2064,32.25,11,0,3.12,SA,1622095808,
277,Saint-Pierre,-21.3393,55.4781,25.90,68,0,1.54,RE,1622095820,
297,Oruzgan,32.8333,66.0000,29.41,9,0,1.08,AF,1622095829,


In [53]:
# Parameters
target_search = "hotel"
target_radius = 5000
target_type = "lodging"



# set up a parameters dictionary
params = {
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key}


In [66]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['locations'] = f'{lat},{lng}'
    
    # Run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Add hotel's name to the empty column
    try:
        hotel_df.loc[index, 'Hotel Name'] = {results[0]['name']}
    except(KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = 'No lodging nearby'
        
hotel_df

/Users/karlaflores/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date,Hotel Name
47,Miandrivazo,-19.5167,45.4667,23.45,57,0,0.90,MG,1622095634,No lodging nearby
67,Sawtell,-30.3833,153.1000,19.01,55,0,4.12,AU,1622095642,No lodging nearby
68,Miepherkeim,38.1419,41.0081,22.16,23,0,1.20,TR,1622095642,No lodging nearby
128,Mossamedes,-15.1961,12.1522,23.25,58,0,0.96,AO,1622095665,No lodging nearby
134,San Juan,-31.5375,-68.5364,21.52,36,0,0.76,AR,1622095419,No lodging nearby
163,São Filipe,14.8961,-24.4956,22.93,77,0,4.67,CV,1622095680,No lodging nearby
221,Gachsaran,30.3586,50.7981,31.38,13,0,1.75,IR,1622095795,No lodging nearby
242,Sakakah,29.9697,40.2064,32.25,11,0,3.12,SA,1622095808,No lodging nearby
277,Saint-Pierre,-21.3393,55.4781,25.90,68,0,1.54,RE,1622095820,No lodging nearby
297,Oruzgan,32.8333,66.0000,29.41,9,0,1.08,AF,1622095829,No lodging nearby


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…